In [3]:
import gym
import keras
import tensorflow as tf
from keras import layers
import numpy as np

2022-12-16 11:12:54.011919: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 11:12:54.166249: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-16 11:12:54.170190: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 11:12:54.170203: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

# Apprentissage

In [65]:
class DQLearning:

    def __init__(self):
        self.gamma = 0.9
        self.epsilon = 0.8
        self.env = gym.make("CartPole-v1", render_mode=None)
        self.env.reset()
        self.model = keras.Sequential([
            layers.Input(shape=self.env.observation_space.shape),
            layers.Dense(64, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(32, activation='relu'),
            layers.Dense(self.env.action_space.n, activation=None),
        ])
        self.model.summary()
        self.model.compile(optimizer='adam', loss='mse')

    def policy(self, s):
        """
        Applique la politique
        :return:
        """
        random = np.random.random()
        if random < self.epsilon:
            # Random
            return self.env.action_space.sample()
        else:
            return np.argmax(self.model.predict(s))

    def train(self, nb_episode: int):
        """
        Entrainement
        :param nb_epochs:
        :return:
        """
        while nb_episode > 0:
            # Générer une action selon la politique
            action = self.policy(np.reshape(self.env.state, (1, 4)))

            # Joue l'action
            next_state, reward, terminate, truncated, infos = self.env.step(action)
            # Partie terminée ou cassée
            if terminate or truncated:
                print("Terminate or truncated\nReseting game")
                self.env.reset()
                nb_episode -= 1
                continue

            old_value = self.model.predict(np.reshape(self.env.state, (1, 4)))[0]

            next_value = np.max(
                self.model.predict(
                    np.reshape(next_state, (1, 4))))

            # Equation Bellman
            equation = reward + self.gamma * next_value
            old_value[action] = equation
            # Train on batch
            self.model.train_on_batch(np.array([self.env.state]), np.array([old_value]))


In [66]:
qLearning.model(np.reshape(qLearning.env.state, (1, 4)))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.05678957, 0.03035368]], dtype=float32)>

In [67]:
qLearning = DQLearning()
qLearning.train(10)

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_63 (Dense)            (None, 64)                320       
                                                                 
 dropout_21 (Dropout)        (None, 64)                0         
                                                                 
 dense_64 (Dense)            (None, 32)                2080      
                                                                 
 dense_65 (Dense)            (None, 2)                 66        
                                                                 
Total params: 2,466
Trainable params: 2,466
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 74ms/step
action  0
1/1 [==============================] - 0s 22ms/step
[ 0.00988439 -0.04859348]
1/1 [==============================] - 0s 1

KeyboardInterrupt: 